#   区分每一个contig的状态是clustered, overlap还是unassigned

In [1]:
import pandas as pd
import numpy as np

gbg = pd.read_csv("./genome_by_genome_overview_exclude5kb.csv")
gbg_modify = gbg.drop("Unnamed: 0",axis=1)
#ntw = pd.read_csv("/home/disk4-8T/chengzhanwen/onefour_meta_10g/v+m_vcontact2/vcontact_result2/c1.clusters")

for v in gbg_modify.Genome:
#     if 'Overlap' in (gbg.loc[gbg['Genome']== v,'VC Status'].values[0]):
#         vc = gbg.loc[gbg['Genome']== v,'VC Status'].values[0].split()[1][1:-1]
#         print(vc)
#         gbg_modify.loc[gbg_modify['Genome']== v,'VC'] = vc
#         gbg_modify.loc[gbg_modify['Genome']== v,'VC Status'] = 'Overlap'
    if (gbg.loc[gbg['Genome']== v,'VC Status'].values[0]) in ['Clustered','Clustered/Singleton']:
        gbg_modify.loc[gbg_modify['Genome']== v,'VC'] = gbg.loc[gbg['Genome']== v,'VC Subcluster'].values[0]
        gbg_modify.loc[gbg_modify['Genome']== v,'VC Status'] = (gbg.loc[gbg['Genome']== v,'VC Status'].values[0])
    else:
        gbg_modify.loc[gbg_modify['Genome']== v,'VC Status'] = (gbg.loc[gbg['Genome']== v,'VC Status'].values[0])

# 鉴定VC属于哪一个genus 

In [ ]:
import pandas as pd
import numpy as np

gbg = pd.read_csv("genome_by_genome_overview_exclude5kb.csv")
cluster = pd.read_csv("viral_cluster_overview.csv")
for VC in cluster.VC:
    MEM = cluster.loc[cluster['VC'] == VC, 'Members'].values[0]
    MEM = MEM.strip().split(',')
    MEM = [x for x in MEM if 'NODE' not in x]
    #print(VC)
    if not MEM:
        #print(VC)
        cluster.loc[cluster['VC'] == VC, 'V_Order'] = 'Unassigned'
        cluster.loc[cluster['VC'] == VC, 'V_Family'] = 'Unassigned'
        cluster.loc[cluster['VC'] == VC, 'V_Genus'] = 'Unassigned'
    else:
        VC_df = pd.DataFrame(index=['order', 'family', 'genus'])
        for mem in MEM:
            VC_df.loc['order',mem] = gbg.loc[gbg['Genome'] == mem, 'Order'].values[0]
            VC_df.loc['family',mem] = gbg.loc[gbg['Genome'] == mem, 'Family'].values[0]
            VC_df.loc['genus',mem] = gbg.loc[gbg['Genome'] == mem, 'Genus'].values[0]
        VC_df=VC_df.T
        #print(VC_df)
        order_count_max = VC_df['order'].value_counts().idxmax()
#         print(order_count_max)
        VC_df_subfamily = VC_df.where(VC_df['order']==order_count_max)
        family_count_max = VC_df_subfamily['family'].value_counts().idxmax()
#         print(family_count_max)
        VC_df_subgenus = VC_df_subfamily.where(VC_df_subfamily['family']==family_count_max)
        genus_count_max = VC_df_subgenus['genus'].value_counts().idxmax()
#         print(genus_count_max)
#         ####determine which order show highest frequency
#    print('order_counts',VC_df['order'].value_counts())
#    order_count_max = VC_df['order'].value_counts().idxmax()
#    cluster.loc[cluster['VC'] == VC, 'V_Order'] = order_count_max
        cluster.loc[cluster['VC'] == VC, 'V_Order'] = order_count_max
        cluster.loc[cluster['VC'] == VC, 'V_Family'] = family_count_max
        cluster.loc[cluster['VC'] == VC, 'V_Genus'] = genus_count_max
        
cluster.to_csv("./viral_cluster.txt",sep='\t',index=False)

In [2]:
gbg_modify

,Genome,Order,Family,Genus,VC,VC Status,Size,VC Subcluster,VC Subcluster Size,Quality,Adj P-value,Topology Confidence Score,Genera in VC,Families in VC,Orders in VC,Genus Confidence Score
0,Achromobacter~phage~83-24,Caudovirales,Siphoviridae,Jwxvirus,VC_0_0,Clustered,2.0,VC_0_0,2.0,0.1848,0.952724,0.1761,1.0,1.0,1.0,1.0000
1,Achromobacter~phage~JWAlpha,Caudovirales,Podoviridae,Jwalphavirus,VC_8_0,Clustered,20.0,VC_8_0,20.0,0.5921,1.000000,0.5921,7.0,1.0,1.0,0.8316
2,Achromobacter~phage~JWF,Caudovirales,Siphoviridae,Unassigned,VC_17_0,Clustered,2.0,VC_17_0,2.0,1.0000,1.000000,1.0000,2.0,1.0,1.0,1.0000
3,Achromobacter~phage~JWX,Caudovirales,Siphoviridae,Jwxvirus,VC_0_0,Clustered,2.0,VC_0_0,2.0,0.1848,0.952724,0.1761,1.0,1.0,1.0,1.0000
4,Achromobacter~phage~phiAxp-1,Caudovirales,Siphoviridae,Unassigned,VC_1_0,Clustered,12.0,VC_1_0,12.0,0.7363,1.000000,0.7363,5.0,1.0,1.0,1.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5582,Xanthomonas~virus~CP1,Caudovirales,Siphoviridae,Xp10virus,NaN,Overlap (VC_78/VC_131),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5583,Xanthomonas~virus~OP1,Caudovirales,Siphoviridae,Xp10virus,NaN,Overlap (VC_78/VC_131),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5584,Xanthomonas~virus~Xop411,Caudovirales,Siphoviridae,Xp10virus,NaN,Overlap (VC_78/VC_131),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5585,Xanthomonas~virus~Xp10,Caudovirales,Siphoviridae,Xp10virus,NaN,Overlap (VC_78/VC_131),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
